In [2]:
# imports
from itertools import islice
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import pprint
from riotwatcher import LolWatcher, ApiError
my_region = 'na1'
from api_key import API_KEY

lol_watcher = LolWatcher(API_KEY)

In [3]:
# select my riot puuid
my_region = 'na1'
me = lol_watcher.summoner.by_name(my_region, 'slayer56429')
puid = me["puuid"]
print(puid)

XixSHIdrUeBgVQHvWxw8nXK3yY_ybNbj5oeR36b5Db4T93o35j1bJ1sxgO1l1QW4DDHJyDTSnL4mjQ


In [4]:
# find last 100 ranked games - start time currently indicates latest season at the end of 2022
match_hist = lol_watcher.match.matchlist_by_puuid(my_region,puid,count=100, queue = 420,start_time = 1668582000)

In [14]:
# pretty print last game data
last = lol_watcher.match.by_id(my_region,match_hist[0])
# all participant data
participants = last["info"]["participants"]

part_num = 0
for p in participants:
    print(f"---------------------------------------Participant # {part_num}--------------------------------------------")
    part_num+=1
    pprint.pprint(p)
# team data 
teams = last["info"]["teams"]
for t in teams:
    pprint.pprint(t)    

---------------------------------------Participant # 0--------------------------------------------
{'allInPings': 0,
 'assistMePings': 0,
 'assists': 7,
 'baitPings': 0,
 'baronKills': 0,
 'basicPings': 0,
 'bountyLevel': 0,
 'challenges': {'12AssistStreakCount': 0,
                'abilityUses': 213,
                'acesBefore15Minutes': 0,
                'alliedJungleMonsterKills': 0,
                'baronTakedowns': 0,
                'blastConeOppositeOpponentCount': 0,
                'bountyGold': 0,
                'buffsStolen': 0,
                'completeSupportQuestInTime': 0,
                'controlWardsPlaced': 0,
                'damagePerMinute': 1379.883348084213,
                'damageTakenOnTeamPercentage': 0.41352222416286555,
                'dancedWithRiftHerald': 0,
                'deathsByEnemyChamps': 9,
                'dodgeSkillShotsSmallWindow': 5,
                'doubleAces': 0,
                'dragonTakedowns': 1,
                'earliestDragonTak

In [9]:
# define pull current season function which is set to grab the my last 30 games of the newest season which started on Sep 16 2022
current_season_start = []
opps_user_match_data = []
runes = []

def pull_current_season():
    # create list for data to be turned into DataFrames later in function 
    current_season_start = []
    runes = []

    # for loop grabbing all data which pertains to my user last X games  
    for m in range(len(match_hist)):
        game = lol_watcher.match.by_id(my_region,match_hist[m])
        # remove rune data and input into list for later DataFrame usage
        for g in game["info"]["participants"]:
            runes.append(g.pop('perks', None))
            # if statement specifically for pulling my user data based on my riot puuid 
            if puid in g['puuid']:
                self_game_data = g
                # add matchID for later joining usage 
                match_id = game['metadata']['matchId']
                self_game_data['matchID'] = match_id
                current_season_start.append(self_game_data)

            # repeat process for opponent data 
            if puid not in g['puuid']:
                opp_game_data = g
                # add matchID for later joining usage 
                match_id = game['metadata']['matchId']
                opp_game_data['matchID'] = match_id
                opps_user_match_data.append(opp_game_data)
                
    # load data into CSV format excluding certain columns which I'm not interested in yet 
    current_season_df = pd.DataFrame(current_season_start)
    current_season_df = current_season_df.drop(columns=['riotIdName','riotIdTagline','challenges','puuid','summonerId'],axis=1)
    current_season_df.to_csv('../data/current_season_first_30.csv',index=False)
    # repeat for opponent data 
    opp_data = pd.DataFrame(opps_user_match_data)
    opp_data = opp_data.drop(columns=['riotIdName','riotIdTagline','challenges','puuid','summonerId'],axis=1)
    opp_data.to_csv('../data/opp_data.csv',index=False)

In [5]:
opps_user_match_data = []
runes = []

def pull_opps_data():
    
    for m in range(5):
        game = lol_watcher.match.by_id(my_region,match_hist[m])
        
        for g in game["info"]["participants"]:
            runes.append(g.pop('perks', None))
        
            if puid not in g['puuid']:
                opp_game_data = g
                # add matchID for later joining usage 
                match_id = game['metadata']['matchId']
                opp_game_data['matchID'] = match_id
                opps_user_match_data.append(opp_game_data)
                
    opp_data = pd.DataFrame(opps_user_match_data)
    opp_data = opp_data.drop(columns=['riotIdName','riotIdTagline','challenges','puuid','summonerId'],axis=1)
    opp_data.to_csv('../data/opp_data.csv',index=False)

In [8]:
pull_current_season()

In [11]:
data = pd.read_csv("../data/opp_data.csv")
pd.set_option('display.max_columns', None)
data

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,championName,championTransform,commandPings,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,dangerPings,deaths,detectorWardsPlaced,doubleKills,dragonKills,eligibleForProgression,enemyMissingPings,enemyVisionPings,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,getBackPings,goldEarned,goldSpent,holdPings,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,needVisionPings,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,onMyWayPings,participantId,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,profileIcon,pushPings,quadraKills,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win,matchID
0,0,0,7,0,0,0,0,15374,16,14,Sion,0,0,2,12731,15074,12731,82522,0,10,0,2,0,True,0,0,False,False,False,False,False,False,0,12712,12400,0,TOP,0,0,3,3084,3748,3083,3181,0,0,3340,18,2,7,TOP,0,3,2,248,19765,9097,37855,0,12,0,1,0,0,0,0,1,0,155478,30520,47646,3159,0,0,SOLO,0,81,65,55,12,9,14,3,12,606,Teregen,False,100,TOP,56,1867,183328,42956,0,86547,3094,0,188,553,381,1,0,8084,3339,1044,4,5,9,0,0,9,0,1,3,False,NA1_4496541649
1,1,5,3,0,0,0,0,11643,14,777,Yone,0,7,2,357,3036,357,22887,0,8,1,1,0,True,4,0,False,False,False,False,False,False,5,10893,10325,0,MIDDLE,0,0,3,1055,6673,3006,3031,3123,3086,3340,20,4,8,MIDDLE,550,2,2,374,24344,5470,14963,0,32,0,1,0,0,0,9,3,0,124951,13858,11636,1670,2,0,SOLO,0,241,58,45,10,6,14,4,4,519,Sandhu23,False,100,MIDDLE,23,1867,157906,22622,0,27264,3472,0,135,108,257,1,0,8609,3293,664,0,1,9,0,0,27,1,1,9,False,NA1_4496541649
2,0,0,4,0,0,0,0,16315,16,114,Fiora,0,8,1,4118,5084,4118,27498,0,4,0,0,0,True,2,0,False,True,False,False,False,False,0,14854,15230,0,BOTTOM,0,0,3,3153,3074,6632,3111,6333,0,3340,28,3,10,BOTTOM,683,4,1,769,1662,969,13540,0,20,0,1,0,0,0,7,4,0,183939,19121,19428,1589,1,0,CARRY,0,168,18,67,7,7,6,5,4,203,TWMU Ting,False,100,BOTTOM,15,1867,203200,28039,0,34033,13832,2381,242,106,169,10,0,17597,7947,1065,1,2,9,0,0,14,0,1,8,False,NA1_4496541649
3,0,0,14,0,0,0,0,10842,13,235,Senna,0,29,4,2783,10083,2783,11649,0,9,2,0,0,True,0,0,True,False,False,False,False,False,1,9664,9200,0,UTILITY,0,0,3,2015,3864,3179,3006,6672,3086,3364,18,0,2,BOTTOM,449,0,1,563,240,141,8230,0,0,0,1,0,0,0,0,5,0,61105,20570,13951,4022,0,0,SUPPORT,0,67,38,20,8,6,7,5,4,190,QimiZC,False,100,UTILITY,28,1867,68111,22524,855,22525,13011,6162,43,215,265,5,0,6764,1811,344,2,4,9,0,0,77,2,6,32,False,NA1_4496541649
4,0,17,10,0,0,0,3,16742,17,36,DrMundo,0,0,3,9338,14932,9338,50828,0,5,2,0,0,True,21,1,False,False,False,True,False,False,0,13021,11450,0,TOP,0,2,0,1054,3084,3068,3075,3082,3047,3340,22,1,5,TOP,31,3,1,602,92076,27601,12298,0,20,1,0,1,0,0,14,6,0,103000,16855,35066,986,0,0,SOLO,0,179,51,103,9,3,12,6,6,52,infiSKY,False,200,TOP,15,1867,195081,44457,0,51171,19880,0,167,283,140,1,0,4,0,3806,3,5,7,0,0,23,2,3,8,True,NA1_4496541649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,